In [1]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import spacy
from bertopic import BERTopic
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_20newsgroups

model = SentenceTransformer('all-mpnet-base-v2')

/home/marcuswrrn/Projects/Semantic_Quantification/Semantic_Comparison/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/marcuswrrn/Projects/Semantic_Quantification/Semantic_Comparison/.venv/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/marcuswrrn/Projects/Semantic_Quantification/Semantic_Comparison/.venv/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argumen

In [2]:
df = pd.read_json("/home/marcuswrrn/Projects/Semantic_Quantification/Semantic_Comparison/Data_Processing/Data/job_processed_data_main.json")

df.head(20)

,Job_Titles,Neutral,Positive,Negative,Prompt_Tokens,Response_Tokens,Total_Tokens,Neutral_Encodings,Positive_Encodings,Negative_Encodings,Neutral_Terminated,Neutral_Terminated_Encodings,Positive_Leading,Positive_Leading_Encodings,Negative_Leading,Negative_Leading_Encodings,Positive_Topic,Negative_Topic
0,Technical Support Representative,I am a Technical Support Representative,I am knowledgeable and skilled in providing te...,I may encounter difficult and frustrated custo...,62,29,91,"[0.052540335800000004, -0.0741394237, -0.02414...","[0.0398985222, -0.039076753000000006, -0.03879...","[0.0750338361, 0.055929131800000004, -0.034461...",I am a Technical Support Representative.,"[0.0467863977, -0.0582639724, -0.0304021016000...",I am knowledgeable and skilled in providing te...,"[0.0400350429, -0.0512237102, -0.0361330472000...",I may encounter difficult and frustrated custo...,"[0.06889098140000001, 0.053461976300000005, -0...",0,0
1,A&H Underwriter,I am an A&H Underwriter,I have a great job as an A&H Underwriter.,I am not happy with my job as an A&H Underwriter.,65,35,100,"[-0.0183508322, 0.025549562600000002, -0.02162...","[-0.0315244012, 0.0867658779, -0.0072295303, -...","[-0.027420181800000002, 0.0644791871, -0.00949...",I am an A&H Underwriter .,"[-0.022203857100000002, 0.0287630353, -0.02035...",I have a great job as an A&H Underwriter,"[-0.0302970372, 0.0902502164, -0.0129407654, -...",I am not happy with my job as an A&H Underwriter,"[-0.026900822300000002, 0.0706868768, -0.01072...",-1,1
2,A&H Underwriting Manager,I am an A&H Underwriting Manager,I have a high-level position as an A&H Underwr...,"I am not just an entry-level employee, but I h...",65,44,109,"[-0.017012547700000002, 0.0080151027, -0.02787...","[-0.0027675340000000003, 0.068974562, -0.00189...","[-0.0194694344, 0.0587330759, -0.0063610324, -...",I am an A&H Underwriting Manager.,"[-0.0162954777, 0.0157897249, -0.0263584778000...",I have a high-level position as an A&H Underwr...,"[-0.0147530027, 0.0676713511, 0.0007165152, -0...","I am not just an entry-level employee, but I h...","[-0.0198444836, 0.054371766700000004, -0.00303...",0,0
3,A&P Technician,I am an A&P Technician,Being an A&P Technician is a rewarding career ...,The job of an A&P Technician can be physically...,63,55,118,"[-0.040869303, -0.0479970723, -0.0461201258, -...","[-0.027997929600000002, 0.0399987362, -0.03595...","[-0.0378840007, -0.030123831700000003, -0.0423...",I am an A&P Technician .,"[-0.0417694934, -0.0297952704, -0.0478198938, ...",Being an A&P Technician is a rewarding career ...,"[-0.028020246000000002, 0.0396952815, -0.03876...",The job of an A&P Technician can be physically...,"[-0.0376795791, -0.030884342300000003, -0.0492...",0,0
4,ABA Therapist,I am an ABA Therapist,I provide effective therapy to children with a...,I do not enjoy working as an ABA Therapist and...,63,37,100,"[0.0211608782, -0.0087794699, -0.0511314794, -...","[0.014801985600000001, 0.0252737999, -0.049951...","[0.0058508902, -0.0159540195, -0.0475744903, -...",I am an ABA Therapist.,"[0.015887849000000002, 0.0088945329, -0.048975...",I provide effective therapy to children with a...,"[0.0150047606, 0.0090112872, -0.05012279, 0.01...",I do not enjoy working as an ABA Therapist and...,"[0.0030206721000000003, -0.0278182309, -0.0552...",0,-1
5,ABAP Developer,I am an ABAP Developer,I have expertise in ABAP development and can c...,I do not have experience in other programming ...,62,44,106,"[0.015295145100000001, 0.0369808078, -0.054170...","[0.0101113003, 0.07414821540000001, -0.0528797...","[-0.0060721342, 0.0074372943, -0.0438291319, -...",I am an ABAP Developer.,"[0.0141174691, 0.051188509900000005, -0.056231...",I have expertise in ABAP development and can c...,"[0.0095568541, 0.060523558400000003, -0.052691...",I do not have experience in other programming ...,"[-0.0057134763, 0.0113966065, -0.0453605466, -...",0,-1
6,AEM Architect,I am an AEM Architect,I have extensive knowledge and experience in A.

## GPT-4 Description of Topic 0 for positive and negative sentence pairs using a random sample of 50 sentences as given context

### Positive
All of the sentences provided describe roles, responsibilities, skills, or expertise that someone has or is gaining in a specific field or position. Each sentence provides insight into what the individual does (roles/responsibilities), what they know or are learning (skills/knowledge/expertise), or the impact of their work. The specific fields and roles vary widely, including customer service, data management, construction, healthcare, software development, and more. However, they are all linked by this common theme of describing professional responsibilities or abilities.

Topic 0: roles and responsibilities, with an emphases on skills, knowledge, expertise or impact of their work

### Negative
All of these sentences discuss some form of challenge, stress, or difficulty related to a particular job or role. These challenges could be physical (like demanding physical labor or long hours), emotional (like dealing with difficult customers or situations), or systemic (such as strict regulations, bureaucracy, or additional responsibilities). The sentences articulate the complexities and hardships faced by individuals in various professional or personal roles.

Topic 0: challenges, stress or difficulty related to the role

## Human understanding of additional negative clusters
Topic 1, appears to discuss a lack of fulfillment and/or satisfation working in a role. Sentences appear to follow very similar structure. All remaining topics demonstrate domain specific expertise. For example Topic 2 is medical orientated and discusses a lack of qualification to perform surgeries or give medical advice to people. Topic 3 is structured similarily except it is focused on the lack of qualification to give legal and financial advice

In [3]:
def column_mean(list_of_lists):
    # Convert the input list of lists to a numpy array
    data = np.array(list_of_lists)
    # Compute the mean average along the columns (axis=0)
    return np.mean(data, axis=0)
    
def get_embedding(df: pd.DataFrame, topic_num, is_positive=True):
    encodings = None
    if is_positive:
        encodings = df[df["Positive_Topic"] == topic_num]["Positive_Encodings"].to_list()
    else:
        encodings = df[df["Negative_Topic"] == topic_num]["Negative_Encodings"].to_list()
    return [float(x) for x in column_mean(encodings)]

In [4]:
# Get the negative clusters
negative_0 = get_embedding(df, 0, is_positive=False)
negative_1 = get_embedding(df, 1, is_positive=False)
negative_neg1 = get_embedding(df, -1, is_positive=False)
# Cluster positive topics together
positive_0 = get_embedding(df, 0)
positive_neg1 = get_embedding(df, -1)

# Generate the base embeddings
positive = [float(x) for x in column_mean(df["Positive_Encodings"].to_list())]
negative = [float(x) for x in column_mean(df["Negative_Encodings"].to_list())]

In [5]:
# Use neutral sentences to create a base 'job' embedding
# This embeddings anchors the topic to help determine whether the topic of the sentence is based around working
job_sentences = [
    "Having a job provides financial stability.",
    "Employment offers opportunities for personal growth.",
    "A job allows individuals to contribute to society.",
    "Having a job can provide a sense of purpose.",
    "Employment can foster a sense of belonging.",
    "A job offers the chance to develop new skills.",
    "Having a job can provide a social network.",
    "Employment offers a structured routine.",
    "A job allows individuals to gain experience.",
    "Having a job can lead to professional advancement.",
    "Employment provides access to employee benefits.",
    "A job offers opportunities for career development.",
    "Having a job can provide a sense of accomplishment.",
    "Employment allows individuals to support their families.",
    "A job offers the chance to make meaningful contributions.",
    "Having a job can provide a sense of identity.",
    "Employment provides an opportunity to learn from colleagues.",
    "A job allows individuals to contribute to their community.",
    "Having a job can foster personal and professional growth.",
    "Employment offers a platform to utilize one's skills.",
    "A job offers a source of income.",
    "Having a job can enhance self-esteem.",
    "Employment provides opportunities for networking.",
    "A job allows individuals to develop their strengths.",
    "Having a job can lead to new opportunities.",
    "Employment offers a sense of stability.",
    "A job provides a platform for continuous learning.",
    "Having a job can promote work-life balance.",
    "Employment offers a sense of structure.",
    "A job allows individuals to make a difference.",
    "Having a job can provide a sense of belonging.",
    "Employment provides access to professional development.",
    "A job offers opportunities for personal and professional fulfillment.",
    "Having a job can lead to increased self-confidence.",
    "Employment allows individuals to contribute to the economy.",
    "A job provides a platform for teamwork.",
    "Having a job can foster a sense of independence.",
    "Employment offers opportunities for goal achievement.",
    "A job allows individuals to build a professional network.",
    "Having a job can enhance one's skill set.",
    "Employment provides a platform for creativity.",
    "A job offers opportunities for problem-solving.",
    "Having a job can provide a sense of structure.",
    "Employment allows individuals to learn from diverse perspectives.",
    "A job provides an opportunity to develop leadership skills.",
    "Having a job can lead to personal and professional fulfillment.",
    "Employment offers a platform to apply academic knowledge.",
    "A job allows individuals to overcome challenges.",
    "Having a job can foster a sense of camaraderie.",
    "Employment provides opportunities for professional growth.",
    "A job offers the chance to learn from experienced professionals.",
    "Having a job can promote a healthy work-life balance.",
    "Employment allows individuals to contribute to the success of an organization.",
    "A job provides a platform to utilize one's talents.",
    "Having a job can enhance problem-solving skills.",
    "Employment offers opportunities for personal achievement.",
    "A job allows individuals to develop transferable skills.",
    "Having a job can lead to financial independence.",
    "Employment provides access to professional networks.",
    "A job offers the chance to learn from diverse perspectives.",
    "Having a job can foster creativity.",
    "Employment allows individuals to build a professional reputation.",
    "A job provides opportunities for personal development.",
    "Having a job can promote teamwork and collaboration.",
    "Employment offers a platform for career progression.",
    "A job allows individuals to contribute to the growth of an organization.",
    "Having a job can foster a sense of responsibility.",
    "Employment provides access to training and educational opportunities.",
    "A job offers opportunities for mentorship.",
    "Having a job can enhance communication skills.",
    "Employment allows individuals to develop time management skills.",
    "A job provides a platform for innovation.",
    "Having a job can lead to professional recognition.",
    "Employment offers a sense of purpose and direction.",
    "A job allows individuals to contribute to the well-being of others.",
    "Having a job can foster a sense of empowerment.",
    "Employment provides opportunities for continuous learning.",
    "A job offers a source of personal and professional growth.",
    "Having a job can promote diversity and inclusion.",
    "Employment allows individuals to apply their knowledge and expertise.",
    "A job provides a platform to build a professional reputation.",
    "Having a job can enhance leadership skills.",
    "Employment offers opportunities for skill development.",
    "A job allows individuals to pursue their passions.",
    "Having a job can lead to increased job satisfaction.",
    "Employment provides access to professional resources.",
    "A job offers the chance to learn from industry experts.",
    "Having a job can foster a sense of community.",
    "Employment allows individuals to develop resilience.",
    "A job provides opportunities for personal growth and fulfillment.",
    "Having a job can promote creativity and innovation.",
    "Employment offers a platform for knowledge sharing.",
    "A job allows individuals to contribute to societal progress.",
    "Having a job can enhance problem-solving abilities.",
    "Employment provides access to career advancement opportunities.",
    "A job offers opportunities for personal and professional development.",
    "Having a job can lead to increased job security.",
    "Employment allows individuals to build a professional portfolio.",
    "A job provides a platform to apply theoretical knowledge in practice.",
    "Having a job can foster a sense of achievement and success.",
    "Employment offers a sense of belonging and purpose.",
    "A job allows individuals to learn from constructive feedback.",
    "Having a job can promote work-life integration.",
    "Employment provides opportunities for personal and professional connections."
]
neutral = [float(x) for x in column_mean(model.encode(job_sentences))]

In [6]:
def evaluate_cluster_scores(sentence):
    # Encode the sentence
    encoding = model.encode(sentence)
    print(f"\n{sentence}")
    print("+========================================+")
    print(f"Positive 0 Similarity (responsibilities + abilities): {util.cos_sim(encoding, positive_0)[0][0]}")
    print(f"Positive -1 Similarity: {util.cos_sim(encoding, positive_neg1)[0][0]}")
    print("+========================================+")
    print(f"Negative 0 Similarity: {util.cos_sim(encoding, negative_0)[0][0]}")
    print(f"Negative 1 Similarity (Job Satisfaction): {util.cos_sim(encoding, negative_1)[0][0]}")
    print(f"Negative -1 Similarity: {util.cos_sim(encoding, negative_neg1)[0][0]}")
    print("+========================================+")
    print(f"Full Positive: {util.cos_sim(encoding, positive)[0][0]}")
    print(f"Full Negative: {util.cos_sim(encoding, negative)[0][0]}")
    print(f"Neutral Anchor: {util.cos_sim(encoding, neutral)[0][0]}\n")

In [7]:
# Evaluating non-job related sentences
evaluate_cluster_scores("The world is so amazing!")
evaluate_cluster_scores("The world is so awful!")
evaluate_cluster_scores("The movie was great!!! I LOVE THIS DIRECTOR!!!")


The world is so amazing!
+========================================+
Positive 0 Similarity (responsibilities + abilities): 0.05918579548597336
Positive -1 Similarity: 0.037502650171518326
+========================================+
Negative 0 Similarity: -0.02982320450246334
Negative 1 Similarity (Job Satisfaction): -0.05140244960784912
Negative -1 Similarity: -0.1122361421585083
+========================================+
Full Positive: 0.05598362535238266
Full Negative: -0.10048113763332367
Neutral Anchor: 0.011398880742490292


The world is so awful!
+========================================+
Positive 0 Similarity (responsibilities + abilities): -0.013604339212179184
Positive -1 Similarity: -0.02466106414794922
+========================================+
Negative 0 Similarity: 0.03482728451490402
Negative 1 Similarity (Job Satisfaction): 0.15412688255310059
Negative -1 Similarity: -0.004902703687548637
+========================================+
Full Positive: -0.01533011719584465
Full 

In [8]:
# Evaluate sentences which reference jobs
evaluate_cluster_scores("I love this director!")
evaluate_cluster_scores("I love this actor!")
evaluate_cluster_scores("I love actors!")


I love this director!
+========================================+
Positive 0 Similarity (responsibilities + abilities): 0.2453722357749939
Positive -1 Similarity: 0.22335731983184814
+========================================+
Negative 0 Similarity: 0.17502319812774658
Negative 1 Similarity (Job Satisfaction): 0.11515495181083679
Negative -1 Similarity: 0.07852836698293686
+========================================+
Full Positive: 0.24269865453243256
Full Negative: 0.10893526673316956
Neutral Anchor: 0.0620652474462986


I love this actor!
+========================================+
Positive 0 Similarity (responsibilities + abilities): 0.17776566743850708
Positive -1 Similarity: 0.16616548597812653
+========================================+
Negative 0 Similarity: 0.08511428534984589
Negative 1 Similarity (Job Satisfaction): 0.01590423285961151
Negative -1 Similarity: -0.0002584792673587799
+========================================+
Full Positive: 0.17656980454921722
Full Negative: 0.02059

In [9]:
evaluate_cluster_scores("I love being an actor!")
evaluate_cluster_scores("I am an actor!")
evaluate_cluster_scores("I am an actor")


I love being an actor!
+========================================+
Positive 0 Similarity (responsibilities + abilities): 0.3360162377357483
Positive -1 Similarity: 0.34205424785614014
+========================================+
Negative 0 Similarity: 0.2270602434873581
Negative 1 Similarity (Job Satisfaction): 0.1458188146352768
Negative -1 Similarity: 0.15435004234313965
+========================================+
Full Positive: 0.33809894323349
Full Negative: 0.185767263174057
Neutral Anchor: 0.258379727602005


I am an actor!
+========================================+
Positive 0 Similarity (responsibilities + abilities): 0.428953617811203
Positive -1 Similarity: 0.4264768958091736
+========================================+
Negative 0 Similarity: 0.26836684346199036
Negative 1 Similarity (Job Satisfaction): 0.1889016032218933
Negative -1 Similarity: 0.24697934091091156
+========================================+
Full Positive: 0.4301334619522095
Full Negative: 0.27260082960128784
Neutra

In [10]:
evaluate_cluster_scores("I clean toilets")
evaluate_cluster_scores("I fly helicopters")
evaluate_cluster_scores("I fly helicopters for a living")
evaluate_cluster_scores("I manage teams")


I clean toilets
+========================================+
Positive 0 Similarity (responsibilities + abilities): 0.5133974552154541
Positive -1 Similarity: 0.526936948299408
+========================================+
Negative 0 Similarity: 0.3799170255661011
Negative 1 Similarity (Job Satisfaction): 0.23482348024845123
Negative -1 Similarity: 0.32122981548309326
+========================================+
Full Positive: 0.517480731010437
Full Negative: 0.36028122901916504
Neutral Anchor: 0.279506653547287


I fly helicopters
+========================================+
Positive 0 Similarity (responsibilities + abilities): 0.5034869909286499
Positive -1 Similarity: 0.5161787271499634
+========================================+
Negative 0 Similarity: 0.3645266592502594
Negative 1 Similarity (Job Satisfaction): 0.22982639074325562
Negative -1 Similarity: 0.3200727701187134
+========================================+
Full Positive: 0.5073133111000061
Full Negative: 0.3557974696159363
Neutral A

In [11]:
evaluate_cluster_scores("I am a Janitor")
evaluate_cluster_scores("I am a Project Manager")
evaluate_cluster_scores("I am an Engineer")
evaluate_cluster_scores("I am a Construction Worker")


I am a Janitor
+========================================+
Positive 0 Similarity (responsibilities + abilities): 0.5445908308029175
Positive -1 Similarity: 0.5768424272537231
+========================================+
Negative 0 Similarity: 0.4733430743217468
Negative 1 Similarity (Job Satisfaction): 0.2828274965286255
Negative -1 Similarity: 0.3946486711502075
+========================================+
Full Positive: 0.551691472530365
Full Negative: 0.4423362612724304
Neutral Anchor: 0.3033313751220703


I am a Project Manager
+========================================+
Positive 0 Similarity (responsibilities + abilities): 0.6170756816864014
Positive -1 Similarity: 0.6374945640563965
+========================================+
Negative 0 Similarity: 0.518903374671936
Negative 1 Similarity (Job Satisfaction): 0.4607018828392029
Negative -1 Similarity: 0.558711051940918
+========================================+
Full Positive: 0.6223013401031494
Full Negative: 0.5935977101325989
Neutral A

In [12]:
# Code used to randomly sample sentences from topic clusers
import random

def select_random_values(original_list, num_values):
    # Check if the number of values requested is greater than the original list length
    if num_values > len(original_list):
        num_values = len(original_list)
    
    # Shuffle the original list
    random.shuffle(original_list)
    
    # Select the first 'num_values' elements
    selected_values = original_list[:num_values]
    
    return selected_values


positive_text = df[df["Positive_Topic"] == 1]["Positive"].to_list()
negative_text = df[df["Negative_Topic"] == 5]["Negative"].to_list()

positive_vals = select_random_values(positive_text, 50)
negative_vals = select_random_values(negative_text, 50)

In [13]:
for text in negative_vals:
    print(f"\n{text}")


I lack knowledge in other fields as I am solely focused on Information Security as a Specialist.

I lack experience in other areas of design beyond digital.

I lack experience in other areas of data analysis beyond digital data.

I am not proficient in other programming languages and may not be suitable for roles outside of Java development.

I lack experience in other programming languages and technologies outside of Full Stack Javascript as a Software Developer.

I lack experience in other areas of IT architecture beyond cloud infrastructure.

I am not familiar with other programming languages and may not be able to work on projects that require knowledge of other languages.

I am not skilled in other areas of design beyond user experience.

I lack experience in other fields of data analysis.

I may not have expertise in other areas of IT beyond networking.

I lack experience in other areas of data management and analysis.

I lack experience in other areas of IT architecture.

I may